In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model

In [4]:
train=[]
train_y=[]

for i in range(1,5000):
    train.append(pd.read_csv('./masked_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    train_y.append(pd.read_csv('./dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())

train = np.array(train)
train_y = np.array(train_y)

In [ ]:
train=[]
train_y=[]

for i in range(1,3200):
    train.append(pd.read_csv('./masked_single_label_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    train_y.append(pd.read_csv('./single_label_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())

train = np.array(train)
train_y = np.array(train_y)

In [4]:
multi_train=[]
multi_train_y=[]

for i in range(1,3200):
    multi_train.append(pd.read_csv('./imsi_mask/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    multi_train_y.append(pd.read_csv('./imsi/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())

multi_train = np.array(multi_train)
multi_train_y = np.array(multi_train_y)

In [3]:
train = train.swapaxes(1,2)
train_y = train_y.swapaxes(1,2)

In [5]:
model = tf.keras.models.Sequential()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-14 21:37:24.894892: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-14 21:37:24.895292: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [41]:
# model.add(Input(shape=(9,32)))

# inputs = Input(shape=(9,32))
# lstm =  tf.keras.layers.Bidirectional(LSTM(units=32, return_sequences=True, activation='sigmoid'))(inputs)
# outputs = tf.keras.layers.Bidirectional(LSTM(units=32, return_sequences=False, activation='sigmoid'))(lstm)
# forward = LSTM(units=16, return_sequences=True, activation='sigmoid')
# backward = LSTM(units=16, return_sequences=True, activation='sigmoid', go_backwards=True)

# model.add(tf.keras.layers.Bidirectional(forward, backward_layer=backward, input_shape=(9,32)))
# model.add(LSTM(units=32, return_sequences=True, activation='sigmoid', input_shape=(9,32)))
# model.add(LSTM(units=9, return_sequences=True, activation='sigmoid', input_shape=(32,9)))
model.add(LSTM(units=22, activation='softmax', input_shape=(32,22)))
# model = Model(inputs, outputs)

In [42]:
# model.compile(loss = tf.keras.losses.binary_crossentropy, optimizer='rmsprop')
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer='adam')
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
model.fit(train,train_y, epochs=4000, callbacks=callback)

In [5]:
test_x=[]
test_y=[]

for i in range(3201,3556):
    test_x.append(pd.read_csv('./masked_single_label_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    test_y.append(pd.read_csv('./single_label_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    
test_x = np.array(test_x)
test_y = np.array(test_y) 

In [6]:
multi_test_x=[]
multi_test_y=[]

for i in range(3201,3556):
    multi_test_x.append(pd.read_csv('./masked_single_label_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    multi_test_y.append(pd.read_csv('./single_label_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    
multi_test_x = np.array(multi_test_x)
multi_test_y = np.array(multi_test_y) 

In [8]:
test_x = test_x.swapaxes(1,2)
test_y = test_y.swapaxes(1,2)

In [10]:
multi_dec = np.zeros_like(multi_train)

for i in range(multi_dec.shape[0]):
    for j in range(multi_dec.shape[1]):
        if j != 0:
            multi_dec[i][j] = multi_train_y[i][j-1]

In [6]:
dec = np.zeros_like(train)

for i in range(dec.shape[0]):
    for j in range(dec.shape[1]):
        if j != 0:
            dec[i][j] = train_y[i][j-1]

In [12]:
multi_decn = np.zeros_like(multi_test_y)
for i in range(multi_decn.shape[0]):
    for j in range(multi_decn.shape[1]):
        if j != 0:
            multi_decn[i][j] = multi_test_y[i][j-1]

In [13]:
decn = np.zeros_like(test_y)
for i in range(decn.shape[0]):
    for j in range(decn.shape[1]):
        if j != 0:
            decn[i][j] = test_y[i][j-1]

In [ ]:
asdf = model.predict(test_x)

asdf = asdf.round()

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
i = 10
print(test_x[i])
print('-'*10)
print(asdf[i].astype(int))
print('-'*10)
print(test_y[i])

--

In [7]:
encoder_inputs = Input(shape=(32,23))
encoder_lstm = LSTM(units=512, return_state=True, dropout=0.1, recurrent_dropout=0.1)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [8]:
decoder_inputs = Input(shape=(None,23))
decoder_lstm = LSTM(units=512, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.1)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(23, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [18]:
# if model is not None:
del model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MSE)

In [21]:
with tf.device('/gpu:0'):
    model.fit(x=[train, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.2)

Epoch 1/5000


KeyboardInterrupt: 

In [78]:
model = tf.keras.models.load_model('LSTM_EnDe_Flip_testloss02806.h5')

In [19]:
model.evaluate(x=[test_x, decn], y=test_y)

12/12 [==============================] - 1s 75ms/step - loss: 0.0105


0.010490505024790764

In [20]:
asd = model.predict(x=[test_x, decn])
asd = asd.round()

In [29]:
i = 195
print(test_x[i].astype(int))
print('-'*10)
print(asd[i].astype(int))
print('-'*10)
print(test_y[i].astype(int))

[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 

In [61]:
model.save('LSTM_EnDe_testloss3225.h5')

In [21]:
encoder_model = Model(encoder_inputs, encoder_states)

In [22]:
decoder_state_input_h = Input(shape=(512,))
decoder_state_input_c = Input(shape=(512,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [23]:
n_features = 9
n_timesteps_in = 32

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, n_features))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_seq = list()
    while not stop_condition:

        # in a loop
        # decode the input to a token/output prediction + required states for context vector
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # convert the token/output prediction to a token/output
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_digit = sampled_token_index
        # add the predicted token/output to output sequence
        decoded_seq.append(sampled_digit)
        

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_digit == '\n' or
           len(decoded_seq) == n_timesteps_in):
            stop_condition = True

        # Update the input target sequence (of length 1) 
        # with the predicted token/output 
        target_seq = np.zeros((1, 1, n_features))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update input states (context vector) 
        # with the ouputed states
        states_value = [h, c]

        # loop back.....
        
    # when loop exists return the output sequence
    return decoded_seq